In [ ]:
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm
from sklearn.model_selection import train_test_split  
import mediapipe as mp

dataset_path = r"C:\Users\Note_Book\Numbers_Model\Numbers"

save_folder = r"C:\Users\Note_Book\Numbers_Model"

os.makedirs(save_folder, exist_ok=True)

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7)

data = []  
labels = []  
label_dict = {}

all_folders = sorted(os.listdir(dataset_path))

for idx, folder in enumerate(all_folders):
    label_dict[idx] = folder  

for idx, folder in enumerate(all_folders):
    folder_path = os.path.join(dataset_path, folder)  
    label = folder

    for file in tqdm(os.listdir(folder_path), desc=f"Processing '{label}'"):
        img_path = os.path.join(folder_path, file)  
        img = cv2.imread(img_path)  

        if img is None:  
            continue

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        results = hands.process(img_rgb)

        if results.multi_hand_landmarks:  
            for hand_landmarks in results.multi_hand_landmarks:
                x_ = []
                y_ = []
                features = []

                for lm in hand_landmarks.landmark:
                    x_.append(lm.x)
                    y_.append(lm.y)

                for lm in hand_landmarks.landmark:
                    features.append(lm.x - min(x_))  
                    features.append(lm.y - min(y_)) 

                if len(features) == 42:  
                    data.append(features)
                    labels.append(idx)  
        else:
            features = [0] * 42  
            data.append(features)
            labels.append(10) 

data = np.array(data)
labels = np.array(labels)


Processing 'space': 100%|██████████| 1000/1000 [00:25<00:00, 38.67it/s]


In [2]:
from tensorflow.keras.utils import to_categorical

num_classes = len(label_dict)
labels_cat = to_categorical(labels, num_classes=num_classes)


In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data, labels_cat, test_size=0.2, random_state=42, stratify=labels
)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

model = Sequential([
    Dense(128, activation='relu', input_shape=(42,)),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

c:\Users\moham\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [6]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)


Epoch 1/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5542 - loss: 1.6685 - val_accuracy: 0.9673 - val_loss: 0.1840
Epoch 2/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9506 - loss: 0.1963 - val_accuracy: 0.9899 - val_loss: 0.0629
Epoch 3/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9798 - loss: 0.0924 - val_accuracy: 0.9827 - val_loss: 0.0644
Epoch 4/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9794 - loss: 0.0820 - val_accuracy: 0.9913 - val_loss: 0.0373
Epoch 5/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9846 - loss: 0.0573 - val_accuracy: 0.9928 - val_loss: 0.0330
Epoch 6/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9859 - loss: 0.0532 - val_accuracy: 0.9913 - val_loss: 0.0332
Epoch 7/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9896 - loss: 0.0409 - val_accuracy: 0.9952 - val_loss: 0.0320
Epoch 8/10
260/260 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9895 - loss: 0.0433 - val_accuracy: 0.

In [7]:
loss, acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {acc:.2f}")


82/82 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9919 - loss: 0.0245
Test Accuracy: 0.99


In [8]:
from tensorflow.keras.models import load_model
import pickle

model.save("Numbers_model.h5")

In [9]:
lab={}
all_folders = sorted(os.listdir(dataset_path))
for idx, folder in enumerate(all_folders):
    lab[idx] = folder.upper()

with open("numbers_dict.pkl", "wb") as f:
    pickle.dump(lab, f)